# Data Engineer Test (Part 2)

In [1]:
#importing all required libraries ahead of time :)
#easy to spot any libraries not currently not installed in ur machine
%run kaggle.py
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import iqr
import requests
import mpld3
import json
import datetime as dt
from datetime import datetime
from dateutil import parser
import time
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import inflect
%matplotlib inline
mpld3.enable_notebook()
pd.options.display.float_format = '{:,.2f}'.format

## Question 3
Jobposts Data Exploration and Analysis (code in python)<br><br>
a) Reuse code from Q2 to download the following Kaggle dataset:
Jobposts Data: https://www.kaggle.com/madhab/jobposts/<br><br>
b) Extract the following fields from the jobpost column:
1. Job Title
2. Position Duration
3. Position Location
4. Job Description
5. Job Responsibilities
6. Required Qualifications
7. Remuneration
8. Application Deadline
9. About Company<br>

c) Identify the company with the most number of job ads in the past 2 years<br>
d) Identify the month with the largest number of job ads over the years<br>
e) Find median, mean, min and max values for each product<br>
f) Clean text and generate new text from Job Responsibilities column: The new text<br>
shall not contain any stop words, and the plural words shall be converted into<br>
singular words.
g) Store the results in a new Dataframe/SQL table<br>

In [2]:
#Kaggle credentials
# $ cp cred.json.example cred.json
# $ vim cred.json 
cred = json.load(open('cred.json'))

#login first before download
kaggle = Kaggle()
kaggle.login(cred['UserName'],cred['Password'])
local_file = "job_posts.csv.zip"
dataset_url = "https://www.kaggle.com/madhab/jobposts/downloads/data%20job%20posts.csv"
kaggle.download_dataset(dataset_url,local_file)

In [3]:
q3_df = pd.read_csv(local_file)

In [4]:
#let's take a glimpe to first 2 rows
print(q3_df.count()['jobpost'])
q3_df.head(2)

19001


,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False


In [5]:
# nice, Regex should be enough to extract 
# hopefully all rows are consistent, :)
# turns out not all, but seems manageable
print(q3_df['jobpost'][0][0:100])

AMERIA Investment Consulting Company
JOB TITLE:  Chief Financial Officer
POSITION LOCATION: Yereva


In [6]:
def extract_job(text):
    #regex = r"(?:(?:JOB )?TITLE:([\s\S]+?))?(?:(?:POSITION)DURATION:([\s\S]+?))?(?:(?:POSITION)?LOCATION:([\s\S]+?))?(?:(?:JOB)?DESCRIPTION:([\s\S]+?))?(?:(?:JOB)?RESPONSIBILITIES:)"
    #The above code will run faster but have expensive maintainance cost
    # It also cannot handle if the job posting have different position location for the keywords
    # please bear this slowness for the sake of maintainability :)
    job = {
    'title' :'',
    'duration' : '',
    'location' : '',
    'description' : '',
    'responsibility' : '',
    'qualification' : '',
    'remuneration' : '',
    'deadline' : '',
    'about' : ''
    }
    common_reg = r':([\s\S]+?)(?:([A-Z\s/]+:)|---)'
    match = re.search(r'TITLE'+common_reg, text)
    if match:
        job['title'] = match.group(1).strip()
    match = re.search(r'DURATION'+common_reg, text)
    if match:
        job['duration'] = match.group(1).strip()
    match = re.search(r'LOCATION'+common_reg, text)
    if match:
        job['location'] = match.group(1).strip()
    match = re.search(r'DESCRIPTION'+common_reg, text)
    if match:
        job['description'] = match.group(1).strip()
    match = re.search(r'RESPONSIBILITIES'+common_reg, text)
    if match:
        job['responsibility'] = match.group(1).strip()
    match = re.search(r'QUALIFICATIONS'+common_reg, text)
    if match:
        job['qualification'] = match.group(1).strip()
    match = re.search(r'(?:REMUNERATION|REMUNERATION[\s\S]*?SALARY)'+common_reg, text)
    if match:
        job['remuneration'] = match.group(1).strip()
    match = re.search(r'DEADLINE'+common_reg, text)
    if match:
        job['deadline'] = match.group(1).strip()
    match = re.search(r'ABOUT COMPANY'+common_reg, text)
    if match:
        job['about'] = match.group(1).strip()
    return job

#job = extract_job(q3_df['jobpost'][3135])
#print(job['title'])
#print(job['duration'])
#print(job['location'])
#print(job['remuneration'])
#print(job['deadline'])



#### Extraction of job

To speed up the development and testing I only used sample out of 74 of 19001. Full sample size can take up few minutes and will be time consuming. To run for full sample size just simply replcace
```
samples = q3_df.sample(n)
```
into
```
samples = q3_df
```

In [7]:
start = time.time()
count = 0
stats = []
# 99% conf. level
# +- 15% conf. interval
n = 74
samples = q3_df.sample(n)
samples['e_title'] = np.NaN
samples['e_duration'] = np.NaN
samples['e_location'] = np.NaN
samples['e_description'] = np.NaN
samples['e_responsibility'] = np.NaN
samples['e_qualification'] = np.NaN
samples['e_remuneration'] = np.NaN
samples['e_deadline'] = np.NaN
samples['e_about'] = np.NaN
for i,r in samples.iterrows():
    job = extract_job(r['jobpost'])
    samples.loc[i,'e_title'] = job['title']
    samples.loc[i,'e_duration'] = job['duration']
    samples.loc[i,'e_location'] = job['location']
    samples.loc[i,'e_description'] = job['description']
    samples.loc[i,'e_responsibility'] = job['responsibility']
    samples.loc[i,'e_qualification'] = job['qualification']
    samples.loc[i,'e_remuneration'] = job['remuneration']
    samples.loc[i,'e_deadline'] = job['deadline']
    samples.loc[i,'e_about'] = job['about']
    
    count = count + 1
    if count%1000 == 0:
        stop = time.time()
        elapsed = stop-start
        stats.append(elapsed)
        print(elapsed)
    
stop = time.time()
elapsed = stop-start
stats.append(elapsed)
print(elapsed)

0.33917999267578125


In [8]:
#let's inspect by taking sample to make sure everything is in order
#if nothing weird, will assume it's correct
#samples[['e_title']].values
#samples['e_title'].values
#samples['e_duration'].values
#samples['e_location'].values
#samples['e_description'].values
#samples['e_responsibility'].values
#samples['e_qualification'].values
#samples['e_remuneration'].values
#samples['e_deadline']
#samples['e_about']

#### Identify the company with the most number of job ads in the past 2 years
ArmenTel CJSC has the most number of jobs posted from 2014-2015

In [9]:
#df.loc[:,'parsed_date'] = q3_df['date'].apply(lambda x: dt.datetime.strptime(x,"%b %d, %Y"))
q3_df['parsed_date'] = np.NaN
for i,r in q3_df.iterrows():
    try:
        q3_df.loc[i,'parsed_date'] = datetime.strptime(r['date'],"%b %d, %Y")
    except ValueError:
        q3_df.loc[i,'parsed_date'] = parser.parse(r['date'])
        #job posts from 2004 to 2015
        if q3_df.loc[i,'parsed_date'].year == 2017:
            q3_df.loc[i,'parsed_date'] = q3_df.loc[i,'parsed_date'].replace(year=2015)
        

In [10]:
q3_df[q3_df['parsed_date'].dt.year >= 2014].groupby('Company').size().sort_values(ascending=False).head(5)

Company
ArmenTel CJSC                                102
Mentor Graphics Development Services CJSC     90
World Vision Armenia                          71
Career Center NGO                             69
Orange Armenia CJSC                           68
dtype: int64

#### Identify the month with the largest number of job ads over the years
March seems to hold the largest number of job posted while start and end of year have the lowest.

In [11]:
q3_df.groupby(q3_df['parsed_date'].dt.month).size().sort_values(ascending=False)

parsed_date
3     1702
2     1665
6     1662
9     1652
10    1637
8     1613
7     1595
5     1580
11    1573
4     1466
12    1432
1     1424
dtype: int64

#### Clean text and generate new text from Job Responsibilities column: 

The new text shall not contain any stop words, and the plural words shall be converted into
singular words.



In [12]:
new_responsibility = samples['e_responsibility'].copy()

In [13]:
tokenizer = RegexpTokenizer(r'\w+')
p = inflect.engine()
for i,r in samples.iterrows():
    tokens = tokenizer.tokenize(r['e_responsibility'])
    filtered_words = [word for word in tokens if word not in stopwords.words('english')]
    new = ""
    for filtered_word in filtered_words:
        temp = p.singular_noun(filtered_word)
        if temp != False:
            new += temp + " "
        else:
            new += filtered_word + " "
    new_responsibility[i] = new

In [14]:
new_responsibility.shape

(74,)

In [15]:
new_responsibility.head(5).values

array([ 'Implement proactive searching attraction potential client Maintain relationship potential current customer Prepare credit package analysi Track overall lending proces Review loan banking service application Perform regular monitoring credit portfolio Be involved banking service sale ',
       'Research current fashion trend industry person study trend popular Analyze sale pricing datum choose product based research result determine style trend generate sale profit Work merchandising team fashion designer department manager understand product highest marketability bring profit company Evaluate sale report determine item selling checking inventory record place new order necessary Review pricing information different supplier negotiate best price Ensure supplier deliver new stock time Monitor best selling piece store make sure alway available consumer Determine next big hit fashion industry type fashion accessory bag shoe demand upcoming season target market Meet regularly fashio

#### Store the results in a new Dataframe/SQL table

In [16]:
type(new_responsibility)

pandas.core.series.Series

In [17]:
df = new_responsibility.to_frame().reset_index()

In [18]:
type(df)

pandas.core.frame.DataFrame

In [19]:
df.head(5)

,index,e_responsibility
0,10111,Implement proactive searching attraction poten...
1,10645,Research current fashion trend industry person...
2,13696,Research follow legal regulation RA Inform SEF...
3,9493,Program mindset passion deliver state art appl...
4,2995,Find identify right outlet within trade channe...
